# Out of Band Protocol - Sender

The out of band protocol allows agents to exchange messages without requiring a DIDComm channel. This can be used to establish a connection, request a presentation or issue a credential. 

The RFC is described [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0434-outofband)



## 1. Initialise the Controller

In [1]:
%autoawait
import time
import asyncio
import json
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)


IPython autoawait is `on`, and set to use `asyncio`


## 2. Configure Listeners for Connection Webhooks

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    print("Connection Webhook : ", payload)
    
connections_listener = {
    "topic": "connections",
    "handler": connections_handler
}


agent_controller.register_listeners([connections_listener], defaults=True)

## 3. Create an out of band connection invitation

In [3]:
payload = {
  "include_handshake": True,
  "use_public_did": False
}

# Create an out of band Invitation
oob_invite = await agent_controller.oob.create_invitation(payload)
print("COPY OOB INVITE")
print(oob_invite["invitation"])

Connection Webhook :  {'routing_state': 'none', 'accept': 'auto', 'invitation_key': 'GFttRWJAJ7cV1EDS1b3AJatE4nznL6wAH4tdHeh2VXQm', 'updated_at': '2020-11-02 09:45:51.275856Z', 'initiator': 'self', 'invitation_mode': 'once', 'state': 'invitation', 'connection_id': '668c6512-9897-4592-b460-db8cec9007ae', 'created_at': '2020-11-02 09:45:51.275856Z'}
COPY OOB INVITE
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': '8d250512-82b6-4d2f-b593-1dcc76077af5', 'handshake_protocols': ['https://didcomm.org/connections/1.0/invitation', 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation'], 'request~attach': [], 'label': 'Alice', 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6Mkui9w1kYbdf6x7j48hA119gSDtNGdjzBWy5oZ7vf3QkC9'], 'routingKeys': [], 'serviceEndpoint': 'http://192.168.65.3:8020'}]}
Connection Webhook :  {'routing_state': 'none', 'accept': 'auto', 'invitation_key': 'GFttRWJAJ7cV1EDS1b3AJatE4nznL6wAH4tdHeh

## 4. Copy Invitation Across to Bob's [Notebook](http://localhost:8889/notebooks/Part%208%20-%20Out%20of%20Band%20Protocol.ipynb)

## 10. Connection Should Now Be Active

In [4]:
response = await agent_controller.connections.get_connections()
print(response)
connection_id = response["results"][0]["connection_id"]

{'results': [{'routing_state': 'none', 'accept': 'auto', 'invitation_key': 'GFttRWJAJ7cV1EDS1b3AJatE4nznL6wAH4tdHeh2VXQm', 'updated_at': '2020-11-02 09:46:26.668961Z', 'initiator': 'self', 'invitation_mode': 'once', 'my_did': 'fMye9F8QS4eQoSB4QrDNj', 'state': 'active', 'their_label': 'Bob', 'connection_id': '668c6512-9897-4592-b460-db8cec9007ae', 'their_did': 'E418SRJtbFgMGAWiXPao8f', 'created_at': '2020-11-02 09:45:51.275856Z'}]}


## 11. Write schema to ledger

In [11]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_contributor2"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor2:0.0.1


## 12. Write credential definition to ledger

In [12]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default


## 13. Populate the Attribues to Issue to Bob

In [14]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "skill", "value": "researcher"},
    {"name": "age", "value": "21"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 14. Create credential issuance record

In [15]:
payload = {
  "auto_remove": "true",
  "comment": "string",
  "cred_def_id": cred_def_id,
  "credential_proposal": {
    "@type": "issue-credential/1.0/credential-preview",
    "attributes": credential_attributes
  },
  "issuer_did": "PQRXDxdGqQGSZ8z69p4xZP",
  "schema_id": schema_id,
  "schema_issuer_did": "PQRXDxdGqQGSZ8z69p4xZP",
  "trace": "false"
}

# Create a cred issue
cred_create = await agent_controller.issuer.create_credential(payload)
print(cred_create)
credential_exchange_id = cred_create['credential_exchange_id']

{'auto_issue': True, 'role': 'issuer', 'auto_offer': False, 'initiator': 'self', 'credential_exchange_id': 'f95fa8c1-2a21-4f7f-8a03-4fc2753cc938', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor2:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default', 'key_correctness_proof': {'c': '48851036833714144414836034256032760860451746984433888967284304248782808409536', 'xz_cap': '11919714033391301613671038794446651280061680829411228701193596772983224681778983412285003234193262500272368171820934586750503789285615064845490003121425851230585988133786268260605688802704949164249693048825113942688183291395631384098235851441681437896526275191176783952432509492141003830931051698782323545193221427419852091462137277955632120398227241925810464441428650935456852880483267386011994388481974546803806263385807677577038994611771483754226840870558344793144962819084198970964742584862520216751869168059582222443352722014204108435820873984499581891720054694128197802453446

In [16]:
print(credential_exchange_id)

f95fa8c1-2a21-4f7f-8a03-4fc2753cc938


## 15. Send Credential Out of Band

In [22]:
payload = {
  "attachments": [
    {
      "id": credential_exchange_id,
      "type": "credential-offer"
    }
  ],
  "include_handshake": False,
  "use_public_did": False
}

oob_invite_cred_issue = await agent_controller.oob.create_invitation(payload)
print("COPY OOB INVITE")
print(oob_invite_cred_issue["invitation"])

Connection Webhook :  {'connection_id': '9c4f920c-7e8d-4020-8d20-96f61625ab6c', 'state': 'invitation', 'invitation_key': '8p9cgbMsWzQGDD51ysn638DZyTTt3c9ZTPsxUH7FFFro', 'created_at': '2020-11-02 11:32:30.140938Z', 'initiator': 'self', 'routing_state': 'none', 'accept': 'auto', 'invitation_mode': 'once', 'updated_at': '2020-11-02 11:32:30.140938Z'}
COPY OOB INVITE
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': 'c876d97e-ad5f-4a12-8229-768753856abb', 'request~attach': [{'@id': 'request-0', 'mime-type': 'application/json', 'data': {'json': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '15a997ff-509a-46a7-abc1-d2a54d3fa596', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'credential_preview': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'},

In [18]:
response = await agent_controller.issuer.get_records()
print(response['results'])



[{'auto_issue': True, 'role': 'issuer', 'auto_offer': False, 'initiator': 'self', 'credential_exchange_id': 'f95fa8c1-2a21-4f7f-8a03-4fc2753cc938', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor2:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default', 'key_correctness_proof': {'c': '48851036833714144414836034256032760860451746984433888967284304248782808409536', 'xz_cap': '1191971403339130161367103879444665128006168082941122870119359677298322468177898341228500323419326250027236817182093458675050378928561506484549000312142585123058598813378626826060568880270494916424969304882511394268818329139563138409823585144168143789652627519117678395243250949214100383093105169878232354519322142741985209146213727795563212039822724192581046444142865093545685288048326738601199438848197454680380626338580767757703899461177148375422684087055834479314496281908419897096474258486252021675186916805958222244335272201420410843582087398449958189172005469412819780245344

## End of Tutorial

In [13]:
await agent_controller.terminate()